In [ ]:
# Loading the required Libraries.
import _pickle as pickle
import pandas as pd
import numpy as np
from sklearn import preprocessing
import warnings; warnings.simplefilter('ignore')

In [ ]:
pwd()

In [ ]:
Training_Data = 'Dota_2.vp'

In [ ]:
with open(Training_Data, 'rb') as File:
    D = pickle.Unpickler(File)
    Data = D.load()

In [ ]:
len(Data[0])

In [ ]:
# Team 1’s hero IDs of 54400 matches. 
Data[0][0]

In [ ]:
# Team 0’s hero IDs of 54400 matches.
Data[1][0]

In [ ]:
# Team 1’s stats of 54400 matches.
Data[2][0]

In [ ]:
#Team 0’s stats of 54400 matches.
Data[3][0]


In [ ]:
# Match result of 54400 matches.
# Format (Team ID that won the game, duration of the game)
Data[4][0]

In [ ]:
# Merging the data.
Team_0 = []
Team_1 = []
Team_0 = [[[z[i]] + Data[3][e][i]for i,l in enumerate(z)]for e,z in enumerate(Data[1])]
Team_1 = [[[z[i]] + Data[2][e][i]for i,l in enumerate(z)]for e,z in enumerate(Data[0])]
print(len(Team_0))
print(len(Team_1))

In [ ]:
Team_0[0][0]

In [ ]:
for l, m in  enumerate(Data[4]):
    if m[0] == 0:
        for x,y in enumerate(Team_0[l]):
            Team_0[l][x] = Team_0[l][x] + [1] + [l]
    else:
        for x,y in enumerate(Team_0[l]):
            Team_0[l][x] = Team_0[l][x] + [0] + [l]

In [ ]:
Team_0[0][0]

In [ ]:
Team_1[0][0]

In [ ]:
for l, m in  enumerate(Data[4]):
    if m[0] == 1:
        for x,y in enumerate(Team_1[l]):
            Team_1[l][x] = Team_1[l][x] + [1] + [l]
    else:
        for x,y in enumerate(Team_1[l]):
            Team_1[l][x] = Team_1[l][x] + [0] + [l]


In [ ]:
Team_1[0][0]

In [ ]:
# Combining all the Team Data.
All_Data = []
All_Data = [Team_0[i]+Team_1[i] for i,l in enumerate(Team_1)]
len(All_Data)

In [ ]:
# Unwrapping the Team Data into individual rows for Machine Learning Model.
Final_Data = []
Final_Data = [l for z in All_Data for l in z]
len(Final_Data)

In [ ]:
# Creating DataFrame Object.
Dota_2_DF = pd.DataFrame(Final_Data)
Dota_2_DF.head()

In [ ]:
# Renaming the Columns.
Dota_2_DF.columns = ['Hero_ID','Hero_ID_2','Kills','Deaths','Assists','Available_Gold','Last_Hits','Denies','GPM','XPM','Networth','XP','Stat_1','MVP_Votes','Duration','Win/Loss', 'Match_ID']
Dota_2_DF.head(10)

In [ ]:
# Saving the CSV file.
Dota_2_DF.to_csv('Dota_2_Cleaned_Data.csv')


In [ ]:
ID = []
ID = Dota_2_DF[Dota_2_DF['Hero_ID'].notnull()]['Hero_ID'].astype('int')
ID = list(ID)
U_ID=np.asarray(ID)
Unique_ID = np.unique(U_ID)
len(Unique_ID)

In [ ]:
Scaled_Data = pd.DataFrame()
Scaled_DF = pd.DataFrame()
for l in Unique_ID:
    ID_DF =  Dota_2_DF[Dota_2_DF[Dota_2_DF['Hero_ID'].notnull()]['Hero_ID'] == int(l)]
    DF_Transfomed = ID_DF.drop(['Hero_ID', 'Hero_ID_2', 'MVP_Votes', 'Win/Loss', 'Match_ID', 'Duration'], axis=1)
    Val = DF_Transfomed.values
    Min_Max_Scaler = preprocessing.MinMaxScaler()
    Val_Scaled = Min_Max_Scaler.fit_transform(Val)
    Scaled_DF = pd.DataFrame(Val_Scaled)
    Scaled_DF.columns = ['Kills','Deaths','Assists','Available_Gold','Last_Hits','Denies','GPM','XPM','Networth','XP','Stat_1']
    Scaled_DF['Hero_ID']  = list(ID_DF['Hero_ID'].astype(int))
    Scaled_DF['Hero_ID_2']  = list(ID_DF['Hero_ID_2'].astype(int))
    Scaled_DF['MVP_Votes']  = list(ID_DF['MVP_Votes'].astype(int))
    Scaled_DF['Win/Loss']  = list(ID_DF['Win/Loss'].astype(int))
    Scaled_DF['Match_ID']  = list(ID_DF['Match_ID'].astype(int))
    Scaled_DF['Duration']  = list(ID_DF['Duration'].astype(int))
    Scaled_DF = Scaled_DF[['Hero_ID','Hero_ID_2','Kills','Deaths','Assists','Available_Gold','Last_Hits','Denies','GPM','XPM','Networth','XP','Stat_1','MVP_Votes','Duration','Win/Loss', 'Match_ID']]
    Scaled_Data = pd.concat([Scaled_Data, Scaled_DF])
 

In [ ]:
Scaled_Data = Scaled_Data.sort_values(by=['Match_ID'])
Scaled_Data.head(10)

In [ ]:
Average_Data = pd.DataFrame()
Average_DF = pd.DataFrame()

for l in Unique_ID:
    ID_DF =  Dota_2_DF[Dota_2_DF[Dota_2_DF['Hero_ID'].notnull()]['Hero_ID'] == int(l)]
    DF_Transfomed = ID_DF.drop(['Hero_ID', 'Win/Loss', 'Match_ID'], axis=1)
    Win_Loss = list(ID_DF['Win/Loss'])
    Win_Rate = np.sum(Win_Loss)/len(Win_Loss)
    L1 = list(DF_Transfomed.mean(axis = 0))
    Average_DF = pd.DataFrame(L1).T
    Average_DF.columns = ['Hero_ID_2','Kills','Deaths','Assists','Available_Gold','Last_Hits','Denies','GPM','XPM','Networth','XP','Stat_1','MVP_Votes','Duration']
    Average_DF['Hero_ID']  = [l]
    Average_DF['Win_Rate'] = [Win_Rate]
    Average_DF = Average_DF[['Hero_ID','Hero_ID_2','Kills','Deaths','Assists','Available_Gold','Last_Hits','Denies','GPM','XPM','Networth','XP','Stat_1','MVP_Votes','Win_Rate']]
    Average_Data = pd.concat([Average_Data, Average_DF])
 

In [ ]:
Average_Data.tail(10)

In [ ]:
Join = Average_Data[['Hero_ID','Win_Rate']]

In [ ]:
Scaled_Data = Join.merge(Scaled_Data, on = 'Hero_ID')
Scaled_Data = Scaled_Data[['Hero_ID','Hero_ID_2','Kills','Deaths','Assists','Available_Gold','Last_Hits','Denies','GPM','XPM','Networth','XP','Stat_1','MVP_Votes','Duration','Win/Loss','Win_Rate','Match_ID']]
Scaled_Data = Scaled_Data.sort_values(by=['Match_ID'])
Scaled_Data.head(10)

In [ ]:
Average_Scaled_Data = pd.DataFrame()
Average_DF = pd.DataFrame()
Scaled_DF = pd.DataFrame()
for l in Unique_ID:
    ID_DF =  Dota_2_DF[Dota_2_DF[Dota_2_DF['Hero_ID'].notnull()]['Hero_ID'] == int(l)]
    DF_Transfomed = ID_DF.drop(['Hero_ID', 'Win/Loss', 'Match_ID'], axis=1)
    Win_Loss = list(ID_DF['Win/Loss'])
    Win_Rate = np.sum(Win_Loss)/len(Win_Loss)
    Val = DF_Transfomed.values
    Min_Max_Scaler = preprocessing.MinMaxScaler()
    Val_Scaled = Min_Max_Scaler.fit_transform(Val)
    Scaled_DF = pd.DataFrame(Val_Scaled)    
    L1 = list(Scaled_DF.mean(axis = 0))
    Average_DF = pd.DataFrame(L1).T
    Average_DF.columns = ['Hero_ID_2','Kills','Deaths','Assists','Available_Gold','Last_Hits','Denies','GPM','XPM','Networth','XP','Stat_1','MVP_Votes','Duration']
    Average_DF['Hero_ID']  = [l]
    Average_DF['Win_Rate'] = [Win_Rate]
    Average_DF = Average_DF[['Hero_ID','Hero_ID_2','Kills','Deaths','Assists','Available_Gold','Last_Hits','Denies','GPM','XPM','Networth','XP','Stat_1','MVP_Votes','Win_Rate']]
    Average_Scaled_Data = pd.concat([Average_Scaled_Data, Average_DF])
 

In [ ]:
Average_Scaled_Data

In [ ]:
Scaled_Data.head(100)

In [ ]:
Scaled_Data.to_csv('Dota_2_Scaled_Data.csv')

In [ ]:
Average_Data.to_csv('Dota_2_Heroes_Average_Data.csv')

In [ ]:
Average_Scaled_Data.to_csv('Dota_2_Heroes_Scaled_Average_Data.csv')